# Проект по теме "Машинное обучение для текстов"

## Описание проекта

Интернет-сервис запускает функционал общения с клиентами у себя на сайте. Необходимо построить модель детектирования негативных сообщений для дальнейшей обработки сервисной службой.  

### Цели 
- Обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности сообщений.
- Построить модель со значением метрики качества *F1* не меньше 0.75. 


### Инструкция по выполнению проекта
1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.


### Описание данных
Данные находятся в файле `toxic_comments.csv`. 
- **text** - текст комментария,
- **toxic** - целевой признак.

**План выполнения работы:**  
- 1. Подготовка Данных  
- 2. Обучение моделей  
    2.1 Logistic Regression  
    2.2 NB-SVM  
    2.3 Linear SVC  
- 3. Выводы  

# 1. Подготовка

Подключаем библиотеки:
- `pandas` - для работы с таблицами  
- `seaborn` - для визуализации данных
- `display` - для вывода табличных данных
- `sklearn` - инструменты машинного обучения (модели классификации, метрики для исследования качества моделей, разделение данных, предобработка данных)
- `nltk` - для лемматизации и фильтрации стоп-слов

In [76]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score
import torch
import transformers
from tqdm import notebook
# from deeppavlov.core.common.file import read_json
# from deeppavlov import build_model, configs

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загрузим данные из файла, выведем первые 10 строк таблиц для первого взгляда на данные.

In [2]:
comments = pd.read_csv('toxic_comments.csv')
comments.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [3]:
comments.shape

(159571, 2)

In [4]:
comments['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [5]:
print(f"Процент объектов класса 1 к общему объёму датасета: {(sum(comments['toxic']) / len(comments) * 100):.2f}%")

Процент объектов класса 1 к общему объёму датасета: 10.17%


Подготовим данные для векторизации.
- Приведём кодировку символов к Unicode
- Проведём лемматизацию слов с помощью WordNetLemmatizer() из библиотеки `nltk`
- Удалим пунктуацию и лишние пробелы
- Удалим стоп-слова (пока загрузим список, удалять будем в процессе tf-idf векторизации)

In [6]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [7]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [8]:
corpus = comments['text'].values.astype('U')

In [9]:
corpus_lemm = [lemmatize(clear_text(corpus[i])) for i in range(len(corpus))]

In [10]:
stopwords = set(nltk_stopwords.words('english'))

- Разделим данные на тренировочную и тестовую выборки

In [11]:
X_train, X_test, y_train, y_test = train_test_split(corpus_lemm, comments['toxic'], 
                                                    test_size=0.2,
                                                    random_state=42)

In [12]:
print(f"Размер тренировочного корпуса: {len(X_train)}")
print(f"Размер тренировочного корпуса: {len(X_test)}")

Размер тренировочного корпуса: 127656
Размер тренировочного корпуса: 31915


- Проведём векторизацию корпусов с помощью TfidfVectorizer, заодно удалим стоп-слова.
- Попробуем обучить модель без использования n-gramm

In [13]:
tf_idf_vec = TfidfVectorizer(ngram_range=(1,1), stop_words=stopwords,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [14]:
X_train_vec = tf_idf_vec.fit_transform(X_train)

In [15]:
X_test_vec = tf_idf_vec.transform(X_test)

In [16]:
print(f"Размер тренировочного датасета: {X_train_vec.shape}")
print(f"Размер тренировочного датасета: {X_test_vec.shape}")

Размер тренировочного датасета: (127656, 41830)
Размер тренировочного датасета: (31915, 41830)


# 2. Обучение

Найдём метрику accuracy для константной модели. Будем предсказывать все твиты нетоксичными ('toxic'=0)

In [17]:
base_predicts = pd.Series(data=np.zeros((len(y_test))), index=y_test.index, dtype='int16')
base_accuacy = accuracy_score(y_test, base_predicts)
print(f"Accuracy константной модели {base_accuacy:.3f}")

Accuracy константной модели 0.898


### Logistic Regression

- Для начала попробуем обучить модель логистической регрессии. 
- Обучение, подбор гиперпараметров, кросс-валидацию проведём с помощью `GridSearchCV` библиотеки `sklearn`
- Подбирать будем гиперпараметр регуляризации С

In [18]:
parameters = {'C': np.linspace(10, 20, num = 11, endpoint = True),
             'max_iter': [1000]}
lrm = LogisticRegression()
clf = GridSearchCV(lrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
clf.fit(X_train_vec, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   57.4s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:  1.9min finished


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.]),
                         'max_iter': [1000]},
             scoring='f1', verbose=2)

In [19]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.773
Параметр регуляризации для лучшей модели: {'C': 13.0, 'max_iter': 1000}


In [20]:
lrm = LogisticRegression(C=13, max_iter=1000)
lrm.fit(X_train_vec, y_train)
predict = lrm.predict(X_test_vec)
f1_lr = f1_score(y_test, predict)

In [21]:
print(f"Показатель f1 на тестовой выборке: {f1_lr:.3f}")

Показатель f1 на тестовой выборке: 0.782


- Проверим модель на адекватность. Рассчитаем метрику accuracy и сравним её с константной моделью

In [22]:
accuracy_lr = accuracy_score(y_test, predict)
print(f"Accuracy на логистической регрессии {accuracy_lr:.3f}, больше, чем на константной модели")

Accuracy на логистической регрессии 0.960, больше, чем на константной модели


- Показатель f1 на тестовой выборке удовлетворяет условию задачи.

### NB-SVM

- Теперь попробуем модель `NBSVM` (Naive Bayes - Support Vector Machine). В данной задаче будем испльзовать модель `LinearRegression` с алгоритмом оптимизации `solver='liblinear', dual=True`. В таком случае эти модели ведут себя похожим образом.  
(ссылка на идею https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline)

In [23]:
def prob(x, y_i, y):
    p = x[y==y_i].sum(axis=0)
    return (p+1) / ((y==y_i).sum()+1)

In [24]:
r = np.log(prob(X_train_vec, 1, y_train.values) / prob(X_train_vec, 0, y_train.values))
X_train_nb = X_train_vec.multiply(r)
X_test_nb = X_test_vec.multiply(r)

In [25]:
parameters = {'C': np.linspace(1, 11, num = 11, endpoint = True)}
nblrm = LogisticRegression(solver='liblinear', 
                           dual=True, 
                           max_iter = 1000)
clf_nb = GridSearchCV(nblrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)

In [26]:
clf_nb.fit(X_train_nb, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:  1.4min finished


GridSearchCV(cv=5,
             estimator=LogisticRegression(dual=True, max_iter=1000,
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])},
             scoring='f1', verbose=2)

In [27]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf_nb.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf_nb.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.791
Параметр регуляризации для лучшей модели: {'C': 3.0}


In [28]:
nblrm = LogisticRegression(C=3,
                           solver='liblinear', 
                           dual=True,
                           max_iter=1000)
nblrm.fit(X_train_nb, y_train)
predict = nblrm.predict(X_test_nb)
f1_nblr = f1_score(y_test, predict)

In [29]:
print(f"Показатель f1 на тестовой выборке: {f1_nblr:.3f}")

Показатель f1 на тестовой выборке: 0.793


- Данная модель мало отличается от изначальной Логистической регрессии (f1 вырос на 1%)
- Для сравнения попробуем обучить модель `LinearSVC`. Linear Support Vector Classification

### LinearSVC

In [30]:
parameters = {'C': np.linspace(1, 31, num = 7, endpoint = True)}
lsvcm = LinearSVC(max_iter = 1000)
clf_lsvc = GridSearchCV(nblrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)

In [31]:
clf_lsvc.fit(X_train_vec, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:   34.2s finished


GridSearchCV(cv=5,
             estimator=LogisticRegression(C=3, dual=True, max_iter=1000,
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': array([ 1.,  6., 11., 16., 21., 26., 31.])},
             scoring='f1', verbose=2)

In [32]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf_lsvc.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf_lsvc.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.772
Параметр регуляризации для лучшей модели: {'C': 11.0}


In [33]:
lsvcm = LogisticRegression(C=11,
                           max_iter=1000)
lsvcm.fit(X_train_vec, y_train)
predict = lsvcm.predict(X_test_vec)
f1_lsvc = f1_score(y_test, predict)

In [34]:
print(f"Показатель f1 на тестовой выборке: {f1_lsvc:.3f}")

Показатель f1 на тестовой выборке: 0.782


# 3. Выводы

- Данные о токсичности твитов успешно загружены и обработаны:
    - Лемматизация проведена с помощью `WordNetLemmatizer` библиотеки `nltk`
    - Знаки пунктуации, а также лишние пробелы удалены
    - Стоп слова удалены (список взят из библиотеки `nltk`)
    - Корпус векторизован с помощью `TfidfVectorizer`
- На получившихся данных обучены модели: `LogisticRegression`, `NB-SVM`, `LinearSVC` 
  
| Модель             | f1 score |  
|:-------------------|:---------|  
| LogisticRegression | 0.782    |  
| NB-SVM             | 0.793    |  
| LinearSVC          | 0.782    |  
  
- Качество моделей практически одинаково. Разница не более 1%. Максимальный показатель f1 получен для **NB-SVM: 0.793**
- Кросс-валидация моделей и подбор гиперпараметров проводились с помощью GridSearchCV.
- Проверка на адекватность была проведена для модели LogisticRegression.